# Build a neral network

Neural network comprise of layers/modules that perform operations on data. the **torch.nn** namespace provides all the building blocks you need to build your own neural network. Every nodule in Pytorch subclasses the nn.Module. A neral network is a module itseft that consists of other modules(layers). This nested structure allows for building and managing complex architectures easily.

In [1]:
import os 
import torch 
from torch import nn 
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


## Get a hardware device for training

We want to be able to train our model on a hardware accelerator like the GPU, if it is avalable. 

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


## Define the class

We define our neural network by subclassing **nn.Module**, and initialize the neural network layers **__init__**. Every **nn.Module** subclass implements the opreations on input data in the forward method.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)

In [5]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


To use the model, we pass it the input data. This executes the model's forward, along with some background operations. Do not call **model.forward()** directly!

In [7]:
X = torch.rand(1,28,28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5], device='cuda:0')


## Model layers

Let's break down the layers in the FashionMNIST model. to illustrate it, we will take a sample minibatch of 3 image of size 28x28 and see what happens to it as we pass it through the network.

In [11]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten

We initialize **the nn.Flatten** layer to convert each 2D 28x28 image to a contiguous array of 784 pixel values(the minibatch dimention (at dim=0) is maintained)

In [12]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear

the linear layer is a module that applies a linear transformation on the input using it's stored weights and biases.


In [14]:

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1= layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU 

Non-linear activations are what create the complex mappings between the model's inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use **nn.ReLU** between our linear layers, but there's other activations to introduce non-linearity in your model.

In [15]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0179, -0.7705,  0.4236,  0.3678,  0.4369, -0.4024,  0.0778,  0.0455,
          0.1851, -0.1403,  0.4083, -0.6224,  0.0230,  0.3691, -0.0803,  0.4087,
          0.2182,  0.0994, -0.1126, -0.0170],
        [-0.3562, -0.5996,  0.2206,  0.2441,  0.5474, -0.2011, -0.3475,  0.3834,
          0.4442,  0.0854,  0.2600, -0.1356, -0.5149,  0.2147, -0.0661,  0.4066,
         -0.0545,  0.3415, -0.2220,  0.0901],
        [ 0.0677, -0.6044, -0.0613,  0.4792,  0.4671, -0.3671,  0.0632,  0.4846,
          0.2018, -0.0363,  0.4114, -0.0841, -0.5468,  0.3684,  0.0298,  0.6248,
          0.2271,  0.4734, -0.4564,  0.3079]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0000, 0.0000, 0.4236, 0.3678, 0.4369, 0.0000, 0.0778, 0.0455, 0.1851,
         0.0000, 0.4083, 0.0000, 0.0230, 0.3691, 0.0000, 0.4087, 0.2182, 0.0994,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.2206, 0.2441, 0.5474, 0.0000, 0.0000, 0.3834, 0.4442,
         0.0854, 0.2600, 0.0000, 0.0000, 0.2147, 0.000

## nn.Sequential

**nn.Sequential** is an ordered container of modules. the data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like seq_modules. 


In [16]:
seq_modules = nn.Sequential(flatten, layer1, nn.ReLU(), nn.Linear(20,10))

input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

## nn.Softmax

The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0,1] representing the model's predicted densities for each class. dim paremeter indicates the dimension along which the values must sum to 1.

In [17]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model parameters

Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing **nn.Module** automatically tracks all fields defined inside your model object, and make all parameters accessible using your model's **parameters()** or **named_parameters()** method

In [18]:
print("Model structure", model, "\n\n")

for name, param in model.named_parameters():
    print(f'Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n')

Model structure NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0126,  0.0032,  0.0267,  ...,  0.0209,  0.0174, -0.0245],
        [-0.0212,  0.0085,  0.0275,  ...,  0.0100, -0.0052,  0.0090]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0272, -0.0032], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0090, -0.0256, -0.0145,  ..., -0.0167, -0.0407, -0.0223],
        [ 0.0076, -0.0360, -0.0038,  ..., -0.0314, -0.0145, -0.0439]],
       device='cuda:0', 